# Step-by-step evaluation of FVS-A

After evaluating the fast vertex substitution (FVS) algorithm for the ANPCP, it was detected that the auxiliary data structures were not enough to accurately solve the ANPCP because the algorithm was originally designed for the PCP.
Then the same step-by-step evaluation was made for the PCP and it was proved that in that case, FVS works as expected.
This last exercise also helped to better understand the functionality of FVS, making it easier to adapt it for the ANPCP.
Both evaluations can be found as Jupyter notebooks in the current directory.

The following hypothesis was stated when the FVS was evaluated for ANPCP:

> It seems like the group of users that are attracted to $f_i$ needs to be split in 2 groups (and data structures):
> 
> 1. Those that $f_\alpha \leftarrow f_i$
> 2. And those that $f_\alpha \leftarrow f_{\alpha - 1}$
> 
> Also it seems that because the open facilities closer than $f_\alpha$ determine it, more data structures are needed to store their distances too (currently, for $z$ only $d_\alpha$ and $d_{\alpha + 1}$ are considerated).

As a result, a modified algorithm is going to be evaluated now, called FVS-A, with two changes:
- A new variable is introduced for the objective function of case 2, represented as $\bar{x'}$.
- Data structure $z$ has the same size $p$, but now all the $\alpha$-neighbors of every user are going to be updated, not only its center: $f_1, ..., f_\alpha$.

## Initial fixed data

In [ ]:
from models.instance import Instance
from models.solver import Solver
from models.vertex import Vertex

In [ ]:
users = [
    Vertex(0, 5, 80),
    Vertex(1, 43, 34),
    Vertex(2, 22, 70),
    Vertex(3, 80, 10),
    Vertex(4, 9, 11),
    Vertex(5, 60, 75)
]

facilities = [
    Vertex(0, 30, 50),
    Vertex(1, 81, 61),
    Vertex(2, 11, 21),
    Vertex(3, 55, 25)
]

instance = Instance(users, facilities)

- $n = 6$
- $m = 4$
- $p = 3$
- $\alpha = 2$

In [ ]:
solver = Solver(instance, 3, 2)

solver.solution.open_facilities = {0, 2, 3}
solver.solution.closed_facilities = {1}
solver.allocate_all()
solver.update_obj_func()

solver.plot()

The objective function value, $x$, is:

In [ ]:
solver.solution.get_objective_function()

The critical allocation determines $x$:

In [ ]:
solver.solution.critical_allocation

The critical facility $f_*$ is $0$ and the critical user $u_*$ is $3$, and their distance is $x = 64$.

## The algorithm

### Iteration 1

In [ ]:
import sys

best_obj_func = sys.maxsize
best_in = -1
best_out = -1

#### Loop closed facilities

In [ ]:
solver.solution.closed_facilities

If facility $1$ is inserted, $f_i = 1$, its distance to $u_*$ would be

In [ ]:
fi_distance = solver.instance.get_distance(3, 1)
fi_distance

which is less than $x$, $51 < 64$. This means the $x$ can be broken and find a better value, so $f_i$ remains as $1$.

Now let's evaluate `move()`.

##### Find a facility to close or remove from the solution, $f_r$, when $f_i = 1$.

In [ ]:
current_of = 0
r = {0:0, 2:0, 3:0}
z = {0:0, 2:0, 3:0}

###### Loop users

In [ ]:
solver.instance.users_indexes

$u = 0$

In [ ]:
u_fi_dist = solver.instance.get_distance(0, 1)
u_fi_dist

In [ ]:
closests = solver.get_alpha_range_closests(0)
closests

The $\alpha$-th (in this case second, $\alpha = 2$) closest facility of $u$ is index $2$, $f_\alpha(u) = 2$.

$f_i$ is not closer to $u$ than $f_\alpha$:

$d_i > d_\alpha \rightarrow 78 > 59$

So data structures $r$ and $z$ need to be updated by "deleting" $f_\alpha = 2$.

- $r(f_\alpha) \leftarrow \max\{ r(f_\alpha), d_\alpha \}$
- $r(2) \leftarrow \max\{ r(2), d_2 \}$

In [ ]:
r[2]

In [ ]:
closests[2].distance

- $r(2) \leftarrow \max\{ 0, 59 \}$

In [ ]:
r[2] = 59

- $z(f_\alpha) \leftarrow \max\{ z(f_\alpha), \min\{ d_i, d_{\alpha + 1} \} \}$
- $z(2) \leftarrow \max\{ z(2), \min\{ d_i, d_{3} \} \}$

In [ ]:
z[2]

In [ ]:
u_fi_dist

In [ ]:
closests[3].distance

- $z(2) \leftarrow \max\{ 0, \min\{ 78, 74 \} \}$

In [ ]:
min(78, 74)

- $z(2) \leftarrow \max\{ 0, 74 \}$

In [ ]:
z[2] = 74

Next user $u = 1$

In [ ]:
u_fi_dist = solver.instance.get_distance(1, 1)
u_fi_dist

In [ ]:
closests = solver.get_alpha_range_closests(1)
closests

In [ ]:
closests[2]

$f_\alpha = 0$, $d_\alpha = 21$

$d_i > d_\alpha$ so update $r$ and $z$.

In [ ]:
r[0]

In [ ]:
r[0] = 21

In [ ]:
z[0]

In [ ]:
min(47, 35)

In [ ]:
z[0] = 35

Next user $u = 2$.

In [ ]:
u_fi_dist = solver.instance.get_distance(2, 1)
u_fi_dist

In [ ]:
closests = solver.get_alpha_range_closests(2)
closests

$60 > 50$ so update $r$ and $z$.

In [ ]:
r[2]

In [ ]:
max(59, 50)

$r(2)$ doesn't change.

In [ ]:
z[2]

In [ ]:
min(60, 56)

In [ ]:
max(74, 56)

$z(2)$ doesn't change either.

Next user $u = 3$

In [ ]:
u_fi_dist = solver.instance.get_distance(3, 1)
u_fi_dist

In [ ]:
closests = solver.get_alpha_range_closests(3)
closests

This $u$ is $u_*$. This is the case that breaks $x$, the reason to enter this current loop, because $d_\alpha(u) = 64$ is $x$, and $d_i = 51$ is less than that.

In [ ]:
solver.plot()

So update the current objective function $x'$:

- $x' \leftarrow \max\{ x', \max\{ d_i, d_{\alpha - 1} \} \}$

In [ ]:
current_of

- $x' \leftarrow \max\{ 0, \max\{ 51, 29 \} \}$

In [ ]:
max(51, 29)

- $x' \leftarrow \max\{ 0, 51 \}$

In [ ]:
current_of = 51

Due to $d_i < d_\alpha$, data structures $r$ and $z$ don't get updated.

Next user $u = 4$

In [ ]:
u_fi_dist = solver.instance.get_distance(4, 1)
u_fi_dist

In [ ]:
closests = solver.get_alpha_range_closests(4)
closests

$88 > 44$ so update $r$ and $z$.

In [ ]:
r[0]

In [ ]:
max(21, 44)

In [ ]:
r[0] = 44

In [ ]:
z[0]

In [ ]:
min(88, 48)

In [ ]:
max(35, 48)

In [ ]:
z[0] = 48

Next user $u = 5$

In [ ]:
u_fi_dist = solver.instance.get_distance(5, 1)
u_fi_dist

In [ ]:
closests = solver.get_alpha_range_closests(5)
closests

$25 < 50$ so update $x'$

In [ ]:
current_of

In [ ]:
max(25, 39)

In [ ]:
max(51, 39)

$x'$ doesn't change.

###### Find best deletion

All users were compared, let's check the data structures:

In [ ]:
current_of

In [ ]:
r

In [ ]:
z

The largest distance, $g_1$, and the second largest distance, $g_2$, from $r$ are:

In [ ]:
from models.moved_facility import MovedFacility

g1 = MovedFacility(2, 59)
g2 = MovedFacility(0, 44)

Let $j_*$ be the corresponding index of $g_1$: $j_* = 2$.

Find $f_r$:

$$f_r = \min_{j \in S}\{ \max\{ x', z(j), \max_{l \neq j}{r(l)} \} \}$$

Loop open facilities in the solution, $S$, which are also the indexes or keys of $z$ and $r$:

In [ ]:
solver.solution.open_facilities

For $j = 0$, $l \in \{ 2, 3 \}$:

In [ ]:
max(59, 0)

$r(l) = 59$:

In [ ]:
max(51, 48, 59)

Now there's a better $f_r = j = 0$ with $x_* = 59$.

In [ ]:
fr = MovedFacility(0, 59)

For $j = 2$, $l \in \{0, 3\} $

In [ ]:
max(44, 0)

In [ ]:
max(51, 74, 44)

$74 > 59$ so $x_*$ didn't improved and $f_r = 0$ is maintained.

For $j = 3$, $l \in \{0, 2\}$

In [ ]:
max(44, 59)

In [ ]:
max(51, 0, 59)

In [ ]:
solver.plot()

In [ ]:
solver.swap(1, 0)
solver.plot()
solver.solution

In [ ]:
solver.instance.distances

In [ ]:
solver.instance.sorted_distances

The result of inserting $1$ and removing $0$ is totally different of that we should get from data structures $x'$, $r$, and $z$.

It seems like the group of users that are attracted to $f_i$ needs to be split in 2 groups (and data structures):

- Those that $f_\alpha \leftarrow f_i$
- And those that $f_\alpha \leftarrow f_{\alpha - 1}$

Also it seems that because the open facilities closer than $f_\alpha$ determine it, more data structures are needed to store their distances too (currently, for $z$ only $d_\alpha$ and $d_{\alpha + 1}$ are considerated).